<a href="https://colab.research.google.com/github/roh9pil/note/blob/master/SingleLabel_MultiClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

2113536/2110848 [==============================] - 1s 1us/step


In [0]:
import numpy as np
def vectorize_seq(sequences, dimensions=10000):
  results=np.zeros((len(sequences),dimensions))
  for i, s in enumerate(sequences):
    results[i, s] = 1.
  return results

In [0]:
x_train = vectorize_seq(train_data)

In [0]:
x_test = vectorize_seq(test_data)

In [10]:
len(train_labels)

8982

In [0]:
def to_one_hot(labels, dimensions=46):
  results = np.zeros((len(labels), dimensions))
  for i, l in enumerate(labels):
    results[i, l] = 1.
  return results


In [0]:
one_hot_train_labels = to_one_hot(train_labels)
one_hot_test_labels = to_one_hot(test_labels)

In [0]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

In [0]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc']
)

In [0]:
x_val = x_train[-1000:]
partial_x_train = x_train[:-1000]

y_val = one_hot_train_labels[-1000:]
partial_y_train = one_hot_train_labels[:-1000]

In [21]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

Train on 7982 samples, validate on 1000 samples
Epoch 1/20
7982/7982 [==============================] - 1s 151us/step - loss: 2.6823 - acc: 0.5242 - val_loss: 1.8597 - val_acc: 0.6440
Epoch 2/20
7982/7982 [==============================] - 1s 109us/step - loss: 1.4481 - acc: 0.7047 - val_loss: 1.4067 - val_acc: 0.7120
Epoch 3/20
7982/7982 [==============================] - 1s 111us/step - loss: 1.0608 - acc: 0.7821 - val_loss: 1.2348 - val_acc: 0.7360
Epoch 4/20
7982/7982 [==============================] - 1s 109us/step - loss: 0.8342 - acc: 0.8290 - val_loss: 1.1143 - val_acc: 0.7600
Epoch 5/20
7982/7982 [==============================] - 1s 109us/step - loss: 0.6644 - acc: 0.8628 - val_loss: 1.0592 - val_acc: 0.7620
Epoch 6/20
7982/7982 [==============================] - 1s 111us/step - loss: 0.5337 - acc: 0.8923 - val_loss: 1.0092 - val_acc: 0.7840
Epoch 7/20
7982/7982 [==============================] - 1s 111us/step - loss: 0.4310 - acc: 0.9132 - val_loss: 1.0124 - val_acc: 0.7810


In [29]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd

fig = make_subplots(rows=1, cols=2)
fig.update_layout(height=600, title_text="Loss & Accuracy")

df = pd.DataFrame(history.history)

fig.add_trace(go.Scatter(x=df.index, y=df.loss,
                    mode='lines+markers',
                    name='loss'),
                    row=1, col=1
)
fig.add_trace(go.Scatter(x=df.index, y=df.val_loss,
                    mode='lines+markers',
                    name='validation_loss'),
                    row=1, col=1
)
fig.add_trace(go.Scatter(x=df.index, y=df.acc,
                    mode='lines+markers',
                    name='acc'),
                  row=1, col=2
)
fig.add_trace(go.Scatter(x=df.index, y=df.val_acc,
                    mode='lines+markers',
                    name='validation_acc'),
                    row=1, col=2
              )
fig.show()

In [30]:
model.fit(partial_x_train,
                    partial_y_train,
                    epochs=5,
                    batch_size=512,
                    validation_data=(x_val, y_val))

Train on 7982 samples, validate on 1000 samples
Epoch 1/5
7982/7982 [==============================] - 1s 112us/step - loss: 0.1031 - acc: 0.9618 - val_loss: 1.3443 - val_acc: 0.7710
Epoch 2/5
7982/7982 [==============================] - 1s 110us/step - loss: 0.1018 - acc: 0.9604 - val_loss: 1.2423 - val_acc: 0.7840
Epoch 3/5
7982/7982 [==============================] - 1s 110us/step - loss: 0.1029 - acc: 0.9602 - val_loss: 1.2430 - val_acc: 0.7970
Epoch 4/5
7982/7982 [==============================] - 1s 110us/step - loss: 0.0996 - acc: 0.9583 - val_loss: 1.3196 - val_acc: 0.7770
Epoch 5/5
7982/7982 [==============================] - 1s 110us/step - loss: 0.0989 - acc: 0.9607 - val_loss: 1.3087 - val_acc: 0.7720


In [31]:
result = model.evaluate(x_test, one_hot_test_labels)

2246/2246 [==============================] - 0s 83us/step


In [32]:
result

[1.2963789569832656, 0.7760462760925293]

In [34]:
import copy
test_labels_copy = copy.copy(test_labels)
np.random.shuffle(test_labels_copy)
hits_array = np.array(test_labels) == np.array(test_labels_copy)
hits_array

array([False, False, False, ...,  True, False, False])

In [37]:
float(np.sum(hits_array))/len(test_labels)

0.19367764915405164

In [0]:
predictions = model.predict(x_test)

In [40]:
predictions[0]

array([1.10233586e-05, 1.45860810e-07, 6.86490198e-10, 9.28391457e-01,
       7.13855773e-02, 7.00528613e-09, 1.15801205e-11, 4.42056596e-08,
       4.88230908e-05, 6.32870325e-08, 3.44025253e-08, 2.51013967e-06,
       1.64529672e-06, 1.30276999e-06, 3.25199176e-06, 3.40066690e-11,
       1.22374522e-05, 6.09283930e-08, 1.09667353e-09, 8.55089093e-05,
       1.69643081e-05, 1.64750588e-06, 3.44313605e-07, 3.42885755e-06,
       1.76131576e-09, 9.77927243e-07, 4.34197789e-10, 7.34625360e-07,
       7.08387873e-08, 4.47890216e-06, 8.38906089e-07, 2.14475007e-10,
       1.65502536e-06, 2.54306318e-08, 4.41562370e-10, 5.06596853e-09,
       1.53308010e-05, 4.25817376e-07, 6.54294990e-06, 2.57654392e-06,
       7.57508687e-08, 2.65066937e-08, 1.94427547e-11, 1.82769508e-10,
       1.28023939e-10, 1.51357895e-08], dtype=float32)

In [42]:
np.argmax(predictions[0])

3